In [0]:
from delta.tables import *

stage_table_name = "incremental_load.default.orders_stage"
target_table_name = "incremental_load.default.orders_target"

In [0]:
# Read data from staging table
stage_df = spark.read.table(stage_table_name)
#stage_df = spark.read.format("delta").load("/Volumes/incremental_load/default/orders_data/stage_data/")

In [0]:

# Create equivalent target table schema if target table does not exist
if not spark.catalog.tableExists(target_table_name):
    stage_df.write.format("delta").saveAsTable(target_table_name)
else:
    # Perform Delta table merge query for upsert based on tracking_num column
    target_table = DeltaTable.forName(spark, target_table_name)

    # Define the merge condition based on the tracking num column
    merge_condition = "target.tracking_num = stage.tracking_num"
   
    target_table.alias("target")\
        .merge(stage_df.alias("stage"), merge_condition)\
        .whenMatchedDelete()\
        .execute()
    stage_df.write.format("delta").mode("append").saveAsTable(target_table_name)